In [1]:
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf

In [2]:
class Visualize():

    @staticmethod
    def visualize(geneome_obj, name):
        import pydot
        import math
        import matplotlib.pyplot as plt
        graph = pydot.Dot(graph_type='graph',
                          label=f"Candidate {name}",
                          fontsize=16,
                          fontname='Roboto',
                          rankdir="LR",
                          ranksep=10)
        input_graph = pydot.Subgraph(ontsize=16,
                          fontname='Roboto',
                          rankdir="LR",
                          rank="same")
        output_graph = pydot.Subgraph(ontsize=16,
                                     fontname='Roboto',
                                     rankdir="LR",
                                     rank="same")
        colors = {
            "INPUT": "#F5A286",
            "HIDDEN": "#A8CFE7",
            "OUTPUT": "#F7D7A8"
        }

        node_dict = {}
        for node in geneome_obj.get_all_nodes():
    
            node_type = node.get_type().name
            node_color = colors[node_type]
#             print(node.get_id())
            node_obj = pydot.Node(node.get_id(),
                                  label=f"{node.get_id()}",
                                  xlabel="",
                                  xlp=5,
                                  orientation=0,
                                  height=1,
                                  width=1,
                                  fontsize=10,
                                  shape='circle',
                                  style='rounded, filled',
                                  color=node_color,
                                  fontcolor="white")

            node_dict[node.get_id()] = node_obj
            if node_type == "INPUT":
                input_graph.add_node(node_obj)
            elif node_type == "OUTPUT":
                output_graph.add_node(node_obj)
            else:
                graph.add_node(node_obj)

        width = 2
        for innovation in geneome_obj.connections:
            if geneome_obj.connections[innovation].is_expressed():
                src = node_dict[geneome_obj.connections[innovation].get_in_node()]
                dst = node_dict[geneome_obj.connections[innovation].get_out_node()]
                weight = geneome_obj.connections[innovation].get_weight()
                if weight >= 0:
                    color = "#AADFA2"
                else:
                    color = "red"
                edge = pydot.Edge(src, dst,
                                  color=color,
                                  label="",
                                  fontcolor="#979ba1",
                                  fontsize=10)
                graph.add_edge(edge)

        graph.add_subgraph(input_graph)
        graph.add_subgraph(output_graph)

        im = graph.create_png()
        with open(f"{name}.png", "wb") as png:
            png.write(im)

In [3]:
class ConnectionGene:
#     ConnectionGene(self, in_node, out_node, weight, expressed, innovation_number):
    def __init__(self, in_node, out_node, weight, expressed, innovation_number):
        self._in_node = in_node
        self._out_node = out_node
        self._weight = weight
        self._expressed = expressed
        self._innovation_number = innovation_number
        
    def enable(self):
        self._expressed = True

    def disable(self):
        self._expressed = False

    def get_in_node(self):
        return self._in_node

    def get_out_node(self):
        return self._out_node

    def get_weight(self):
        return self._weight

    def is_expressed(self):
        return self._expressed

    def get_innovation_number(self):
        return self._innovation_number

In [4]:
from enum import Enum
import random
import copy

#might be a dictionary in the futuro
class TYPE(Enum):
    INPUT = 0
    HIDDEN = 1
    OUTPUT = 2

In [5]:
class NodeGene:
    def __init__(self, ident, type_select='INPUT'):
        self._id = ident
        self._type = TYPE[type_select]
        
    def get_type(self):
        return self._type
    
    def get_id(self):
        return self._id
    
    def activation_functions(self):
        self.actvation_functions = lambda x: 1/(1+np.exp(-x))

# the class below works

In [83]:
class Genome:
     #initial connections number
                        #can be: 'min', 'max' or 'fc', 'rand'
    def __init__(self, no_inputs, no_outputs, init_connect = 'min'):
        self._init_connections_no = init_connect
        self.connections = {} #list of ConnectionGene objects
        self._nodes = [] #list of NodeGene
        self._latest_innovation_no = 0
        self.no_inputs = no_inputs
        self.no_outputs = no_outputs
        self.create_layer(self.no_inputs) #default type is INPUT
        self.create_layer(self.no_outputs, type_layer='OUTPUT')
        self.get_in_out_nodes_indeces()
        
        self.connect_initial_NN()
    
    def create_layer(self, no_units, type_layer='INPUT'):
        for i in range(no_units):
            no_nodes = self.add_new_node_to_genome(NodeGene(self.get_no_of_nodes(), type_select=type_layer))
    
    def set_init_connections_no(self):
        import numbers
        fully_connected_no = self.no_inputs * self.no_outputs
        
        if isinstance(self._init_connections_no, numbers.Number):
            if self._init_connections_no  == 1:
                print('1 == 100% of possible connections (' \
                      + str(fully_connected_no) + ' connections).')
                self._init_connections_no = 'max'
                return fully_connected_no
            if self._init_connections_no > 0 and self._init_connections_no < 1:
                return int(np.round(self._init_connections_no*fully_connected_no))
            
            elif self._init_connections_no < self.no_outputs:
                print('number of connections below the minimum of all outputs connected' +\
                      '\nReplaced by the no. of outputs (' +str(self.no_outputs) +  ').')
                return self.no_outputs
            
            elif self._init_connections_no > fully_connected_no:
                print('number of connections above fully connected.' + \
                      'Replaced with fully connected.')
                self._init_connections_no = 'max'
                return fully_connected_no
            else:
                return self._init_connections_no
        
        elif (self._init_connections_no =='max' or self._init_connections_no =='fc'):
            return fully_connected_no
        
        elif self._init_connections_no == 'min':
            return self.no_outputs
        
        else:
            return np.random.randint(self.no_outputs, self.no_inputs * self.no_outputs+1)
        
    def get_in_out_nodes_indeces(self):
        nodes_total = list(range(self.get_no_of_nodes()))
        self.input_idxs = nodes_total[0:self.no_inputs]
        self.output_idxs = nodes_total[self.no_inputs:self.no_inputs + self.no_outputs]
    
    def get_no_of_ConnectionGenes(self):
        return len(self.connections)
    
    def get_no_of_nodes(self):
        return len(self._nodes)
    
    def add_new_node_to_genome(self, newnode):
        self._nodes.append(newnode)
        return self.get_no_of_nodes()
    
    def get_all_nodes(self):
        return self._nodes
    
    def add_new_ConnectionGene(self, in_node_id, out_node_id):
        innov = self.get_innovation_no()
        self.connections[innov] = ConnectionGene(in_node_id, out_node_id, \
                                 self.gen_random_weight(), True, innov)
        self.update_innovation_no()
        return len(self.connections)
    
    def get_innovation_no(self):
        return self._latest_innovation_no
        
    def get_gene_from_genome(self, gene_id):
        return self.connections[gene_id]
    
    def update_innovation_no(self):
        self._latest_innovation_no += 1
        
    def nodes_are_reversed(self, node1, node2):
        nod1, nod2 = node1.get_type(), node2.get_type()
        an_input, an_output, a_hidden = TYPE.INPUT, TYPE.OUTPUT, TYPE.HIDDEN
        
        return nod1 == a_hidden and nod2 == an_input \
            or nod1 == an_output and nod2 == a_hidden \
            or nod1 == an_output and nod2 == an_input
    
    def nodes_are_both_input_or_output(self, node1, node2):
        nod1, nod2 = node1.get_type(), node2.get_type()
        an_input, an_output = TYPE.INPUT, TYPE.OUTPUT
        
        return nod1 == an_input and nod2 == an_input \
            or nod1 == an_output and nod2 == an_output \
            
    def connection_exists(self, node1_id, node2_id):
        for innovation in self.connections:
            in_connect_nod = self.connections[innovation].get_in_node()
            out_connect_nod = self.connections[innovation].get_out_node()
            
            if node1_id is in_connect_nod and node2_id is out_connect_nod \
                or node2_id is in_connect_nod and node1_id is out_connect_nod:
                return True
            
        return False
            
    def add_connection_mutation(self, node1, node2):
        import copy
        if self.nodes_are_reversed(node1, node2):
            node1, node2 = node2, node1    
        node_1_id, node_2_id = node1.get_id(), node2.get_id()
        
        if not self.connection_exists(node_1_id, node_2_id) \
            and not self.nodes_are_both_input_or_output(node1, node2):
            self.add_connection_between_nodes(node_1_id, node_2_id)
    
    def gen_random_weight(self, low_bound=-2, up_bound=2):
        return np.random.uniform(low=low_bound, high=up_bound)

    def mutate(self):
        if np.random.uniform(low=0, high=1) <= 0.5:
             self.add_node_mutation()
        else:
            i = np.random.randint(0, self.get_no_of_nodes())
            j = np.random.randint(0, self.get_no_of_nodes())
            while i == j:
                j = np.random.randint(0, self.get_no_of_nodes())
            node1, node2 = self.get_all_nodes()[i], self.get_all_nodes()[j]
            self.add_connection_mutation(node1, node2)
            
    def add_node_mutation(self, type_node='HIDDEN'):
        idx = np.random.randint(0, self.get_no_of_ConnectionGenes())

        new_connect = self.connections[idx]
        new_connect.disable()
        
        in_node_id, out_node_id = new_connect.get_in_node(), new_connect.get_out_node()
        
        new_node = NodeGene(self.get_no_of_nodes(), type_select = type_node)
        self.add_new_node_to_genome(new_node)
        
        self.add_connection_between_nodes(in_node_id, new_node.get_id())
        self.add_connection_between_nodes(new_node.get_id(), out_node_id)
        
    def add_connection_between_nodes(self, in_node_id, out_node_id):
        connection_does_not_exist = not self.connection_exists(in_node_id, out_node_id)
        if connection_does_not_exist:
            self.add_new_ConnectionGene(in_node_id, out_node_id)
        return connection_does_not_exist
    
    def init_NN_is_fully_connected(self):
        return (self._init_connections_no =='max' or self._init_connections_no =='fc')

#     inputs, outputs = build_list_of_FC_possible_connections()
    def get_all_combins_of_ins_and_outs(self):
        inputs = []
        outputs = []
        
        for In in self.input_idxs:
            for Out in self.output_idxs:
                in_node, out_node = self.get_all_nodes()[In], self.get_all_nodes()[Out]

                if self.init_NN_is_fully_connected():
                ####if fully connected
                    self.add_connection_between_nodes(in_node.get_id(), out_node.get_id())
                else: 
                #### min. connections (no. of outputs) OR fraction of the total
                    # OR random no. bewtween min and max
                    inputs.append(in_node), outputs.append(out_node)
        return inputs, outputs

#     input_idx_rand, output_idx_rand = scramble_ins_outs()
    def get_scrambled_list(self, L):
        return random.sample(L, len(L))
    
    def connect_initial_NN(self):
        print('creating initial NN.')
        no_init_connect = self.set_init_connections_no()
        
        if self.init_NN_is_fully_connected():
            print('initial NN is FC with  ' + str(no_init_connect) + ' connections')
        else: # fraction of the total possible connections, or random
            print('initial NN has ' + str(no_init_connect) + ' connections, out of a max of ' \
                          + str(self.no_inputs * self.no_outputs) + '.')
            
        inputs, outputs = self.get_all_combins_of_ins_and_outs()
        
        k = 0 #k iterates through all possible combinations of inputs and ouputs
        #only inputs are scrambled in the first run, to cover all outputs
        #then ouputs are scrambled every time the lists inputs, outputs have been exhausted
        
        inputs = self.get_scrambled_list(inputs)
        
        #loop samples scrambled lists until desired no_init_connect is reached
        while no_init_connect > 0:
            if k >= len(inputs): #randomness ended up getting the same connections 
                # over and over, so restart the search
                k = 0
                print('too many existing connections attempted; scrambling inputs again.')
                outputs = self.get_scrambled_list(outputs)
            
            in_node, out_node = inputs[k], outputs[k]
            
            #tries to connect two nodes; if connection exists, it returns False
            connection_does_not_exist =\
                self.add_connection_between_nodes(in_node.get_id(), out_node.get_id())                

            if connection_does_not_exist: 
                #no_init_connect - 1: accounts for a new connection between in and out nodes
                no_init_connect = no_init_connect - 1
            
            k = k + 1 #increment iterator for in/out nodes list
                
        print()
        print('DONE.')
    
g = Genome(20, 5, init_connect = '')

creating initial NN.
initial NN has 99 connections, out of a max of 100.
too many existing connections attempted; scrambling inputs again.
too many existing connections attempted; scrambling inputs again.
too many existing connections attempted; scrambling inputs again.
too many existing connections attempted; scrambling inputs again.

DONE.


In [84]:
Visualize().visualize(g, "Genome")

# class for initial network

creating initial NN.
initial NN has 12 connections, out of a max of 100.

DONE.


## NodeGene related classes

In [69]:
from enum import Enum
import random
import copy

############################ 
class TYPE(Enum):
    INPUT = 0
    HIDDEN = 1
    OUTPUT = 2
############################ 
class NodeGene:
    def __init__(self, ident, type_select='INPUT'):
        self._id = ident
        self._type = TYPE[type_select]
        
    def get_type(self):
        return self._type
    
    def get_id(self):
        return self._id
    
    def activation_functions(self):
        self.actvation_functions = lambda x: 1/(1+np.exp(-x))
        
############################ 
class NodeGeneOperations:
#     NodeGeneLayer(no_units, type_layer='INPUT')
    def __init__(self, no_units, type_layer='INPUT'):
        self._type_layer = type_layer
        self._no_units = no_units
        self._nodes = [] #list of NodeGene
        self.create_layer()
        
    def create_layer(self):
        for i in range(self._no_units):
            no_nodes = self.add_NodeGene_to_list( \
                       NodeGene(self.get_no_of_nodes(), type_select=self._type_layer) )
            
    def add_NodeGene_to_list(self, newnode):
        self._nodes.append(newnode)
        return self.get_no_of_nodes()
    
    def get_no_of_nodes(self):
        return len(self._nodes)
    
    def get_node(self, idx):
        return self._nodes[idx]
    
    def get_nodes(self):
        return self._nodes
    
    @staticmethod
    def nodes_are_reversed(node1, node2):
        nod1, nod2 = node1.get_type(), node2.get_type()
        an_input, an_output, a_hidden = TYPE.INPUT, TYPE.OUTPUT, TYPE.HIDDEN
        
        return nod1 == a_hidden and nod2 == an_input \
            or nod1 == an_output and nod2 == a_hidden \
            or nod1 == an_output and nod2 == an_input
    
    @staticmethod
    def nodes_are_both_input_or_output(node1, node2):
        nod1, nod2 = node1.get_type(), node2.get_type()
        an_input, an_output = TYPE.INPUT, TYPE.OUTPUT
        
        return nod1 == an_input and nod2 == an_input \
            or nod1 == an_output and nod2 == an_output

In [65]:
N = NodeGeneLayer(8)

In [66]:
N.nodes_are_both_input_or_output(N.get_node(1), N.get_node(0)) 

True

# classes related to connections

In [67]:
class ConnectionGenesDict:
#     ConnectionGenesDict(in_node_id, out_node_id)
    def __init__(self, in_node_id, out_node_id):
        self.connections = {}
        self.add_new_ConnectionGene(in_node_id, out_node_id)
        self._latest_innov_no = 0
        
    def add_new_ConnectionGene(self, in_node_id, out_node_id):
        innov = self._latest_innov_no
        self.connections[innov] = ConnectionGene(in_node_id, out_node_id, \
                                 self.gen_random_weight(), True, innov)
        self._latest_innov_no += 1 #updates innovation number
        return len(self.connections)
    
    def connection_exists(self, node1_id, node2_id):
        for innov in self.connections:
            in_connect_nod = self.connections[innov].get_in_node()
            out_connect_nod = self.connections[innov].get_out_node()
            
            if node1_id is in_connect_nod and node2_id is out_connect_nod \
                or node2_id is in_connect_nod and node1_id is out_connect_nod:
                return True
        return False

    def get_ConnectionGene(self, gene_id):
        return self.connections[gene_id]

    def get_ConnectionGenesDict(self):
        return self.connections
    
    def gen_random_weight(self, low_bound=-2, up_bound=2):
        return np.random.uniform(low=low_bound, high=up_bound)
        
    def add_connection_between_nodes(self, in_node_id, out_node_id):
        connection_does_not_exist = not self.connection_exists(in_node_id, out_node_id)
        if connection_does_not_exist:
            self.add_new_ConnectionGene(in_node_id, out_node_id)
        return connection_does_not_exist

## set initial connections class - maybe working

In [68]:
class SetInitConnections:
        def __init__(self, no_inputs, no_outputs, init_connect_no):
            self._init_connect_no = init_connect_no
            self.actual_no_of_connect = 0
            self.no_inputs, self.no_outputs = no_inputs, no_outputs
            fully_connected_no = self.no_inputs * self.no_outputs
            self.fully_connected = False
            
            import numbers
            # no: 1 := max no. of connections
                # 0 x := max no. of connections
            if isinstance(self._init_connect_no, numbers.Number):
                self.actual_no_of_connect = init_no_scope(self, fully_connected_no)  
                
            elif self.init_NN_is_fully_connected():
                self.actual_no_of_connect = fully_connected_no
                
            elif init_connect_no == 'min':
                self.actual_no_of_connect = self.no_outputs
            else:
                self.gen_rand_no_of_connections(fully_connected_no)

        def init_no_scope(self, fully_connected_no):
            if init_connect_no  == 1:
                print('1 == 100% of possible connections (' \
                      + str(fully_connected_no) + ' connections).')
                self.fully_connected = True
                return fully_connected_no
            elif init_connect_no > 0 and self._init_connect_no < 1:
                return int(np.round(self._init_connect_no*fully_connected_no))
            elif init_connect_no < self.no_outputs:
                print('number of connections below the minimum of all outputs connected' +\
                      '\nReplaced by the no. of outputs (' +str(self.no_outputs) +  ').')
                return self.no_outputs

            elif init_connect_no > fully_connected_no:
                print('number of connections above fully connected.' + \
                      'Replaced with fully connected.')
                self.fully_connected = True
                return fully_connected_no
            else:
                return self._init_connect_no
                    
        def gen_rand_no_of_connections(self, fully_connected_no):
            self.actual_no_of_connect = np.random.randint(self.no_outputs, self.no_inputs * self.no_outputs+1)
            if self.actual_no_of_connect  is fully_connected_no:
                self.fully_connected = True

        def init_NN_is_fully_connected(self):
            if (self._init_connect_no =='max' or self._init_connect_no =='fc'):
                self.fully_connected = True
            return self.fully_connected
            
        def get_no_of_connections(self):
            return self.actual_no_of_connect

# class initial NN - guaranteed broken now

In [88]:

class Initial_NN:
     #initial connections number
                        #can be: 'min', 'max' or 'fc', 'rand'
    def __init__(self, no_inputs, no_outputs, init_connect = 'min'):
        self._init_connect_no = SetInitConnections(no_inputs, no_outputs, \
                                            init_connect).get_no_of_connections()
        self.no_inputs = no_inputs
        self.no_outputs = no_outputs
        
        self.connections = {} #list of ConnectionGene objects
        self._nodes = [] #list of NodeGene
        self._latest_innov_no = 0
        self.create_init_units() #default type is INPUT
        self.connect_initial_NN()
        
    def init_NN_is_fully_connected(self):
        return (self._init_connect_no =='max' or self._init_connect_no =='fc')

    def create_init_units(self):
        self.input_layer = NodeGeneOperations(self.no_inputs, type_layer='INPUT')
        self.input_idxs = list(range(self.input_layer.get_no_of_nodes()))
        L_in = len(self.input_idxs)
        
        self.output_layer = NodeGeneOperations(self.no_outputs, type_layer='OUTPUT')
        self.output_idxs = list(range(L_in, \
                                      L_in + self.output_layer.get_no_of_nodes()))
        self._nodes.extend(self.input_layer.get_nodes())
        self._nodes.extend(self.output_layer.get_nodes())
        
    def get_node(self, idx):
        return self._nodes[idx]
    def get_nodes(self):
        return self._nodes
    
#     inputs, outputs = build_list_of_FC_possible_connections()
    def get_all_combins_of_ins_and_outs(self):
        inputs = []
        outputs = []
        
        for In in self.input_idxs:
            for Out in self.output_idxs:
                in_node, out_node = self.get_node(In), self.get_node(Out)

                if self.init_NN_is_fully_connected():
                ####if fully connected
                    self.add_connection_between_nodes(in_node.get_id(), out_node.get_id())
                else: 
                #### min. connections (no. of outputs) OR fraction of the total
                    # OR random no. bewtween min and max
                    inputs.append(in_node), outputs.append(out_node)
        return inputs, outputs

#     input_idx_rand, output_idx_rand = scramble_ins_outs()
    def get_scrambled_list(self, L):
        return random.sample(L, len(L))
    
    
#     ConnectionGenesDict()
    def connect_initial_NN(self):
        print('creating initial NN.')
        no_init_connect = self._init_connect_no
        
        if self.init_NN_is_fully_connected():
            print('initial NN is FC with  ' + str(no_init_connect) + ' connections')
        else: # fraction of the total possible connections, or random
            print('initial NN has ' + str(no_init_connect) + ' connections, out of a max of ' \
                          + str(self.no_inputs * self.no_outputs) + '.')
            
        inputs, outputs = self.get_all_combins_of_ins_and_outs()
        
        k = 0 #k iterates through all possible combinations of inputs and ouputs
        #only inputs are scrambled in the first run, to cover all outputs
        #then ouputs are scrambled every time the lists inputs, outputs have been exhausted
        inputs = self.get_scrambled_list(inputs)
        
        #loop samples scrambled lists until desired no_init_connect is reached
        while no_init_connect > 0:
            if k >= len(inputs): #randomness ended up getting the same connections 
                # over and over, so restart the search
                k = 0
                print('too many existing connections attempted; scrambling inputs again.')
                outputs = self.get_scrambled_list(outputs)
            
            in_node, out_node = inputs[k], outputs[k]
            
            #tries to connect two nodes; if connection exists, it returns False
            connection_does_not_exist =\
                self.add_connection_between_nodes(in_node.get_id(), out_node.get_id())                

            if connection_does_not_exist: 
                #no_init_connect - 1: accounts for a new connection between in and out nodes
                no_init_connect = no_init_connect - 1
            
            k = k + 1 #increment iterator for in/out nodes list
                
        print()
        print('DONE.')
    
g = Initial_NN(20, 5, init_connect = '')

creating initial NN.
initial NN has 83 connections, out of a max of 100.


AttributeError: 'Initial_NN' object has no attribute 'add_connection_between_nodes'